In [15]:
# !pip install louvain
!pip install --upgrade --no-cache psutil gputil

     |████████████████████████████████| 283 kB 1.1 MB/s eta 0:00:01
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=5820b83f8e853e9769249755e369640b7d9ca726650b93c13dd9b8a8bf8fda03
  Stored in directory: /tmp/pip-ephem-wheel-cache-zdkp4eo2/wheels/ba/03/bb/7a97840eb54479b328672e15a536e49dc60da200fb21564d53
Successfully built gputil
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.5
    Uninstalling psutil-5.9.5:
      Successfully uninstalled psutil-5.9.5


In [17]:
import psutil
import GPUtil
print(psutil.cpu_percent())
print(psutil.virtual_memory().percent)
GPUtil.showUtilization()

0.1
2.7
| ID | GPU | MEM |
------------------


In [1]:
import os
import subprocess
import sys
sys.path.append('..')
from tools.formating.formating import *
from tools.utils.utils import *
from tools.qc.scanpy_qc import *
import numpy as np
import scipy

scanpy==1.9.5 anndata==0.9.2 umap==0.5.4 numpy==1.22.4 scipy==1.9.1 pandas==1.3.5 scikit-learn==1.3.1 statsmodels==0.14.0 igraph==0.10.8 louvain==0.8.1 pynndescent==0.5.10


In [2]:
# input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_germ line.h5ad"
# input = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"
# input = "/ps/ai-ready/data/error/hca_heart_neuronal_raw.h5ad"
input = "/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin.h5ad"
adata = load_anndata(input)
adata

/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin.h5ad


AnnData object with n_obs × n_vars = 9424 × 58870
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissue', 'dendrogram_computational_compartment_assignment', 'dendrogram_consensus_prediction', 'dendrogram_tissue_cell_type', 'donor_colors', 'donor_method_colors', 'hvg', 'method_colors', 'neighbors', 'organ_tissue_colors', 'sex_colors', 'tissue_colors', 'umap'
    obsm: 'X_pca', 'X_scvi', 'X_scvi_umap', 'X_umap'
    layers: 'decontXcounts', 'raw_counts'
    obsp: 'connectivities', 'distances'

In [ ]:
adata.obs

In [ ]:
adata.obs.index.get_loc('TCGACGGGTACGAGCA-1-HCAHeart8287128')

In [ ]:
type(adata.X)

In [ ]:
adata.X.toarray()

In [ ]:
adata.layers['raw_counts'].toarray()

In [ ]:
adata.raw.X.toarray()

In [ ]:
"raw_counts" in adata.layers.keys()

In [ ]:
isinstance(adata.X, csr_matrix)

In [ ]:
np.set_printoptions(threshold=np.inf)
adata.X.toarray()[5,:]

In [ ]:
type(adata.X.toarray())

In [ ]:
isinstance(adata.X.toarray(), np.ndarray)

In [ ]:
np.max(adata.X)

In [ ]:
np.min(adata.X)

In [ ]:
from scipy.sparse import csr_matrix, find
find(adata.X)

In [ ]:
from scipy import stats
stats.describe(adata.X.toarray())

In [ ]:
np.median(adata.X.toarray())

In [ ]:
min_genes=200
min_cells=3
target_sum=1e4
n_neighbors=10
n_pcs=40

In [ ]:
adata.var_names_make_unique() 

# Filtering low quality reads
sc.pp.filter_cells(adata, min_genes=min_genes)
sc.pp.filter_genes(adata, min_cells=min_cells)
# mitochondrial genes
adata.var['mt']=adata.var_names.str.startswith('MT-')
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True)

adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)
adata.obs.mt_outlier.value_counts()

print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")
adata.X

In [ ]:
# adata=adata[adata.obs.n_genes_by_counts < 2500, :]
# adata=adata[adata.obs.pct_counts_mt < 5, :]

scrub = scr.Scrublet(adata.X, expected_doublet_rate = 0.076)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets(min_counts=2, min_cells=3, 
                                                        min_gene_variability_pctl=85, n_prin_comps=30)
adata.obs['predicted_doublets'].value_counts()
# adata=adata[adata.obs.predicted_doublets=="False", :]

sc.pp.normalize_total(adata, target_sum=target_sum)

sc.pp.log1p(adata)

In [ ]:
adata.X

In [ ]:
sc.pp.highly_variable_genes(adata)

In [ ]:
import scrublet as scr
scrub = scr.Scrublet(adata.X, expected_doublet_rate = 0.076)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets(min_counts=2, min_cells=3, 
                                                        min_gene_variability_pctl=85, n_prin_comps=30)
adata.obs['predicted_doublets'].value_counts()

In [ ]:
adata=adata[adata.obs.predicted_doublets==False, :]
adata

In [3]:
adata = run_scanpy_qc(adata)
adata

filtered out 26085 genes that are detected in less than 3 cells
Total number of cells: 9424
Number of cells after filtering of low quality cells: 7243
Preprocessing...
Simulating doublets...
Embedding transcriptomes using PCA...
Calculating doublet scores...
Automatically set threshold at doublet score = 0.33
Detected doublet rate = 1.9%
Estimated detectable doublet fraction = 42.7%
Overall doublet rate:
	Expected   = 7.6%
	Estimated  = 4.3%
Elapsed time: 14.2 seconds
normalizing counts per cell
    finished (0:00:00)
extracting highly variable genes
    finished (0:00:02)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
regressing out ['total_counts', 'pct_counts_mt']
    sparse input is densified and may lead to high memory use
    finished (0:01:00)


AnnData object with n_obs × n_vars = 7243 × 3853
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'doublet_scores', 'predicted_doublets'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissue', 'dendrogram_compu

In [4]:
adata.X

<7243x3853 sparse matrix of type '<class 'numpy.float32'>'
	with 27907279 stored elements in Compressed Sparse Row format>

In [5]:
isinstance(adata.X, np.ndarray)

False

In [6]:
adata = run_dimension_reduction(adata, layer="log10k")
adata

computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:27)
computing neighbors
    using 'X_pca' with n_pcs = 40
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:05)
computing tSNE
    using 'X_pca' with n_pcs = 50
    using sklearn.manifold.TSNE
    finished: added
    'X_tsne', tSNE coordinates (adata.obsm) (0:00:17)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:11)
running Leiden clustering
    finished: found 27 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:01)
running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished: found 22 clusters and added
    'louvain', the cluster labels (adata.obs, categorical) (0:00:00)


AnnData object with n_obs × n_vars = 7243 × 3853
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'doublet_scores', 'predicted_doublets', 'leiden', 'louvain'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissu

In [ ]:
adata.layers["raw_counts"].toarray()

In [ ]:
adata.layers["log10k"] = adata.X

In [ ]:
del adata.layers["X"]
adata

In [ ]:
adata.uns

In [ ]:
adata.obs

In [12]:
adata.obsm['log10k_umap'].shape

(7243, 2)

In [ ]:
adata.raw.X.toarray()

In [ ]:
adata.var.index

In [ ]:
type(adata.obs['total_counts'].tolist())

In [ ]:
adata.obs['n_counts']

In [ ]:
'pct_counts_mt' in adata.obs.columns

In [ ]:
adata.obs_vector('pct_counts_mt')

In [ ]:
train, test = sc_train_test_split(adata, train_fraction=0.8)

In [7]:
train, val, test = sc_train_val_test_split(adata, train_fraction=0.8, validation_fraction=0.1, test_fraction=0.1)

In [8]:
train

AnnData object with n_obs × n_vars = 5794 × 3853
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'doublet_scores', 'predicted_doublets', 'leiden', 'louvain'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissu

In [9]:
val

AnnData object with n_obs × n_vars = 724 × 3853
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'doublet_scores', 'predicted_doublets', 'leiden', 'louvain'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissue

In [10]:
test

AnnData object with n_obs × n_vars = 725 × 3853
    obs: 'organ_tissue', 'method', 'donor', 'anatomical_information', 'n_counts_UMIs', 'n_genes', 'cell_ontology_class', 'free_annotation', 'manually_annotated', 'compartment', 'gender', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'doublet_scores', 'predicted_doublets', 'leiden', 'louvain'
    var: 'gene_symbol', 'feature_type', 'ensemblid', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: '_scvi', '_training_mode', 'cell_ontology_class_colors', 'dendrogram_cell_type_tissue

In [13]:
test.write_h5ad(filename="/ps/ai-ready/data/Clustering/Tabula_Sapiens/TS_Skin_test.h5ad", compression='gzip')

In [ ]:
test.layers["log10k"].shape

In [ ]:
from scipy.sparse import csr_matrix
test.X = csr_matrix(test.X)
test.X

In [ ]:
test.layers["raw_counts"].toarray().shape

In [ ]:
adata.var

In [ ]:
list(adata.layers.keys())

In [ ]:
adata.n_vars

In [ ]:
adata.n_obs

In [ ]:
adata.obs_names.to_list()

In [ ]:
adata.var_names.to_list()

In [ ]:
list(adata.obsm.keys())

In [ ]:
adata.X

In [ ]:
type(adata.obs_keys())

In [ ]:
adata.var_keys()

In [ ]:
random_state=0
fraction=0.8
np.random.seed(random_state)
old_n_obs = adata.n_obs
old_n_obs
new_n_obs = int(fraction * old_n_obs)
obs_indices = np.random.choice(old_n_obs, size=new_n_obs, replace=False)
obs_indices

In [ ]:
test=adata[obs_indices].copy()
test

In [ ]:
test.var

In [ ]:
original_indices=np.arange(old_n_obs)
original_indices

In [ ]:
diff=np.setdiff1d(original_indices, obs_indices)
diff

In [ ]:
len(diff)

In [ ]:
np.random.shuffle(diff)
diff

In [ ]:
train=adata[diff].copy()
train

In [ ]:
train.obs